In [1]:
%pip install requests
%pip install attrs
%pip install ratelimit
%pip install python-dotenv

import requests
import attr

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import dotenv_values
env = dotenv_values("urbiotica.env")

ENDPOINT = "http://api.urbiotica.net/v2"
ORGANISM = env['ORGANISM']
USERNAME = env['USERNAME']
PASSWORD = env['PASSWORD']

In [3]:
import time
from collections import deque

@attr.s
class Bucket(object):

    # timestamps of last burst
    burst = attr.ib()
    # interval (seconds) of each burst
    interval = attr.ib()
    # max bucket size
    limit = attr.ib()
    # requests.Session
    session = attr.ib()

    @classmethod
    def new(cls, interval, limit):
        return cls(deque(), interval, limit, requests.Session())

    def wait(self):
        now = time.monotonic()
        if len(self.burst) >= self.limit:
            first = self.burst.popleft()
            while (now < first + self.interval):
                # Sleep 1 second more than needed, to
                # compensate for jitters etc.
                time.sleep((first + self.interval) - now + 1)
                now = time.monotonic()
        self.burst.append(now)
        # Devuelvo requests para que sea facil encadenar...
        return self.session


In [20]:
from datetime import datetime, timedelta
from operator import itemgetter, attrgetter
import math

@attr.s
class Api(object):

    endpoint = attr.ib()
    organism = attr.ib()
    token = attr.ib()
    bucket = attr.ib()

    @classmethod
    def login(cls, endpoint, organism, username, password):
        # La API está limitada a 100 peticiones por minuto
        bucket = Bucket.new(60, 100)
        auth = bucket.wait().get(f'{endpoint}/auth/{organism}/{username}/{password}')
        if auth.status_code != 200:
            raise ValueError('Invalid username of password')
        return cls(endpoint, organism, auth.text.strip('"'), bucket)

    def projects(self):
        url = f'{self.endpoint}/organisms/{self.organism}/projects'
        prj = self.bucket.wait().get(url, headers={ 'IDENTITY_KEY': self.token })
        if prj.status_code != 200:
            raise ValueError(f'Failed to retrieve projects for organism {self.organism}')
        return { item['projectid']: Project.new(self, item) for item in prj.json() }

    def _query(self, projectid, path, attrib):
        url = f'{self.endpoint}/organisms/{self.organism}/projects/{projectid}/{path}'
        its = self.bucket.wait().get(url, headers={ 'IDENTITY_KEY': self.token })
        print("DEBUG: URL = ", url)
        if its.status_code != 200:
            raise ValueError(f'Failed to retrieve {path} for project {projectid}')
        return { item[attrib]: item for item in its.json() }


@attr.s
class Project(object):

    api = attr.ib()
    projectid = attr.ib()
    name = attr.ib()
    description = attr.ib()
    timezone = attr.ib()

    @classmethod
    def new(cls, api, project):
        return cls(api, project['projectid'], project['name'], project['description'], project['timezone'])

    def parkings(self):
        return self.api._query(self.projectid, 'parkings', 'pomid')
    
    def spots(self):
        return self.api._query(self.projectid, 'zones', 'zoneid')
    
    def devices(self, zoneid):
        return self.api._query(self.projectid, f'zones/{zoneid}/devices', 'zoneid')

    def rotations(self, pomid, from_dt, to_dt):
        fromiso = datetime.isoformat(from_dt.replace(microsecond=0))
        toiso = datetime.isoformat(to_dt.replace(microsecond=0))
        result = dict()
        poms = self.api._query(self.projectid, f'spots/{pomid}/rotations/finished/{fromiso}/{toiso}', 'pomid')
        rotations = [ {
            'start': datetime.fromisoformat(item['start']),
            'end': datetime.fromisoformat(item['end'])
        } for item in poms[pomid]['rotations'] ]
        return Project._sortby(rotations, 'start')

    def vehicles(self, pomid, from_dt, to_dt):
        fromts = math.floor(from_dt.timestamp())
        tots = math.ceil(to_dt.timestamp())+1
        result = dict()
        poms = self.api._query(self.projectid, f'spots/{pomid}/phenomenons/vehicle_ctrl?start={fromts}&end={tots}', 'pomid')
        measurements = [ {
            'lstamp': datetime.fromtimestamp(int(item['lstamp']) // 1000),
            'value': item['value']
        } for item in poms[pomid]['measurements'] ]
        return Project._sortby(measurements, 'lstamp')

    @staticmethod
    def _sortby(items, field):
        items.sort(key=itemgetter(field))
        return items


In [21]:
api = Api.login(ENDPOINT, ORGANISM, USERNAME, PASSWORD)
api.token

'bk5I90UjkoWScUjpGvoxAg9zpDRvbdg2fL8MS4In6KqvFRaiUllR4aLjZe7xUmhq'

In [22]:
from collections import defaultdict
import itertools

@attr.s
class Timestamp(object):

    date = attr.ib()
    events = attr.ib()

    @classmethod
    def new(cls, date, events):
        return cls(date, events)
    
    def __str__(self):
        return "\n".join(itertools.chain(
            (self.date.isoformat(),),
            (f'  {event}' for event in self.events)
        ))

@attr.s
class Timeline(object):

    events = attr.ib()

    @classmethod
    def new(cls, rotations, vehicles):
        events = defaultdict(list)
        estados = {
            '0': 'ocupada',
            '1': 'libre',
            '-1': 'en estado desconocido',
        }
        for rotation in rotations:
            events[rotation['start']].append("Comienza rotacion")
            events[rotation['end']].append("Termina rotacion de las %s" % rotation['start'])
        for vehicle in vehicles:
            value = vehicle['value']
            events[vehicle['lstamp']].append('Plaza %s (%s)' % (estados[value], value))
        stamps = [ Timestamp.new(date, info) for date, info in events.items() ]
        stamps.sort(key=attrgetter('date'))
        return cls(stamps)
    
    def __str__(self):
        return "\n".join(str(event) for event in self.events)


In [23]:
for prjid, prj in api.projects().items():
    for pomid, superpom in prj.parkings().items():
        for pom in superpom['poms']:
            now = datetime.now()
            bef = now - timedelta(days=1)
            rots = prj.rotations(pom['pomid'], bef, now)
            vehi = prj.vehicles(pom['pomid'], bef, now)
            ts = Timeline.new(rots, vehi)
            print(str(ts))
            break
        break


DEBUG: URL =  http://api.urbiotica.net/v2/organisms/org387236/projects/prj4dd9a3/parkings
DEBUG: URL =  http://api.urbiotica.net/v2/organisms/org387236/projects/prj4dd9a3/spots/45876/rotations/finished/2021-06-09T08:45:00/2021-06-10T08:45:00
DEBUG: URL =  http://api.urbiotica.net/v2/organisms/org387236/projects/prj4dd9a3/spots/45876/phenomenons/vehicle_ctrl?start=1623221100&end=1623307502
2021-06-09T08:45:45
  Plaza ocupada (0)
2021-06-09T09:09:19
  Comienza rotacion
  Plaza libre (1)
2021-06-09T09:23:11
  Termina rotacion de las 2021-06-09 09:09:19
  Plaza ocupada (0)
2021-06-09T09:24:21
  Comienza rotacion
  Plaza libre (1)
2021-06-09T09:25:43
  Termina rotacion de las 2021-06-09 09:24:21
  Plaza ocupada (0)
2021-06-09T09:41:15
  Comienza rotacion
  Plaza libre (1)
2021-06-09T09:43:05
  Termina rotacion de las 2021-06-09 09:41:15
  Plaza ocupada (0)
2021-06-09T10:11:20
  Comienza rotacion
  Plaza libre (1)
2021-06-09T10:17:42
  Plaza ocupada (0)
2021-06-09T10:17:43
  Plaza libre (1)


In [14]:
api.projects()

{'prj4dd9a3': {'projectid': 'prj4dd9a3',
  'organismid': 'org387236',
  'name': 'Alcoy CyD',
  'description': 'EPSG: 25830',
  'timezone': 'Europe/Madrid',
  'attributes': [],
  'types': ['parking'],
  'creation_stamp': 1590144939706}}